In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp import *
from shared_utils import utils

# Formatting the nb
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

# Specific to this project
import A1_utilities as utils

In [ ]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/pmp_dashboard/"

In [ ]:
appropriations_unwanted = []

In [ ]:
ap10 = utils.import_and_clean(
    "AP10 April .xls", "Download", appropriations_unwanted, 10
)

In [ ]:
ap11 = utils.import_and_clean("AP11 May.xls", "Download", appropriations_unwanted, 11)

In [ ]:
ap12 = utils.import_and_clean(
    "AP12 June.xls",
    "Download",
    appropriations_unwanted,
    12,
)

In [ ]:
df1, df2, df3, df4 = utils.pmp_dashboard_sheets(ap12, "22030", "12_2022")

In [ ]:
df4.head()

In [ ]:
int_cols = [
    "ps_alloc",
    "ps_exp",
    "ps_bal",
    "py_pos_alloc",
    "act__hours",
    "oe_alloc",
    "oe_enc",
    "oe_exp",
    "oe_bal_excl_pre_enc",
]

In [ ]:
my_clean_dataframes = []

In [ ]:
div_crosswalks = {
    "State & Fed Mass Trans": "DRMT",
    "Statewide Planning": "DOTP",
    "Research": "DRISI",
    "PSR/PSSR Development": "DOTP",
    "Rail": "DRMT",
    "Planning Administration": "DOTP",
    "Regional Planning": "DOTP",
}

In [ ]:
ap_12_test = import_clean_data("AP12 June.xls", "Download", appropriations_unwanted, 12)

# Fund by Div

In [ ]:
len(df1)

In [ ]:
df1.shape

In [ ]:
df1["Appropriation"].unique()

In [ ]:
df1.head(2)

In [ ]:
df1_cols = [
    "ps_allocation",
    "ps_expenditure",
    "ps_balance",
    "ps_projection",
    "year_expended_pace",
    "ps_%_expended",
    "oe_allocation",
    "oe_encumbrance",
    "oe_expenditure",
    "oe_balance",
    "oe_enc_+_oe_exp_projection",
    "oe_%_expended",
    "total_allocation",
    "total_expenditure",
    "total_balance",
    "total_projection",
    "total_%_expended",
]

In [ ]:
for i in df1_cols:
    print("\n" + i)
    print(ap12[i].sum())
    print(ap_12_test[i].sum())

In [ ]:
ap_12_test.shape

In [ ]:
ap12.shape

# TPSOE Data

In [ ]:
df2.head()

In [ ]:
df2_cols = [
    "allocation",
    "expenditure",
    "balance",
    "encumbrance",
    "projection",
]

In [ ]:
for i in df2_cols:
    print("\n" + i)
    print(df2[i].sum())

# Timeline Data

In [ ]:
len(df3)

In [ ]:
df3["appropriation"].unique()

In [ ]:
timeline_right_order = [
    "appr_catg",
    "fund",
    "fund_description",
    "appropriation",
    "pec_class",
    "pec_class_description",
    "ps_allocation",
    "ps_expenditure",
    "ps_balance",
    "ps_%_expended",
    "ps_projection",
    "py_pos_alloc",
    "act__hours",
    "oe_allocation",
    "oe_encumbrance",
    "oe_expenditure",
    "oe_balance",
    "oe_enc_+_oe_exp_projection",
    "oe_%_expended",
    "total_allocation",
    "total_expenditure",
    "division",
    "total_balance",
    "total_projection",
    "total_%_expended",
    "ap",
]

In [ ]:
df3.columns

In [ ]:
df3_cols = [
    "ps_allocation",
    "ps_expenditure",
    "ps_balance",
    "oe_allocation",
    "ps_projection",
    "oe_expenditure",
    "oe_balance",
    "total_allocation",
    "total_expenditure",
    "total_balance",
    "total_projection",
    "oe_enc_+_oe_exp_projection",
    "total_%_expended",
]

In [ ]:
df3["ap"].value_counts()

In [ ]:
for i in df3_cols:
    print("\n" + i)
    print(df3[i].sum())

In [ ]:
for i in df3_cols:
    print("\n" + i)
    print(ap10[i].sum() + ap11[i].sum() + ap12[i].sum())

In [ ]:
for i in df3_cols:
    print("\n" + i)
    print(ap10[i].sum())

In [ ]:
for i in df3_cols:
    print("\n" + i)
    print(ap11[i].sum())

# PSOE Timeline

In [ ]:
df4.shape

In [ ]:
df4["ap"].value_counts()

In [ ]:
df4_cols = ["allocation", "expense", "balance", "projection", "encumbrance"]